<a href="https://colab.research.google.com/github/huysuy05/LLM_Evals_On_Imbalanced_Datatset/blob/main/eval_with_hgf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install torch numpy pandas transformers bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.8 MB/s eta 0:00:00


In [51]:
import pandas as pd

label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}
imbalanced_data = pd.read_parquet("./ag_news_train_imbalanced.parquet")
balanced_data = pd.read_parquet("./ag_news_train_balanced.parquet")
testing_set = pd.read_parquet("./ag_news_test_small.parquet")

balanced_data["label_word"] = balanced_data["label"].map(label_map)
imbalanced_data["label_word"] = imbalanced_data["label"].map(label_map)

# Shuffle the dataset
imbalanced_data = imbalanced_data.sample(frac=1).reset_index(drop=True)
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)

balanced_data


,text,label,label_word
0,Three survive 11 days under collapsed building...,0,World
1,Roles reverse as chase shifts gears You can se...,1,Sports
2,"Shell, Sinopec in China Retail Venture Royal D...",2,Business
3,Samsung Faces Concerns Over Margins SEOUL (Re...,2,Business
4,Rocket Ship Wins \$10 Million Prize as Private...,3,Sci/Tech
...,...,...,...
3995,Howard records fourth poll win SYDNEY: Austral...,0,World
3996,Netherlands beats Finland 3-1 in World Cup qua...,1,Sports
3997,Shearer Eying Silverware Newcastle hat-trick h...,1,Sports
3998,All hail the invincibles There have been a han...,1,Sports


In [ ]:
!pip install --upgrade bitsandbytes

In [43]:
import pandas as pd


def classify(text, labels, model):
  """
    Use a Pipeline from Transformers for zero shot classification

    Args:
      text (str): The text data per row
      labels (List[int]): A list of label
      model (str): Model name to load
  """
  from transformers import pipeline

  classifier = pipeline("zero-shot-classification", model=model)
  result = classifier(text, labels)
  pred_label = result["labels"][0]
  return pred_label

In [47]:
def evaluate(data, labels, model):
  total_samples = data.shape[0]
  correct_pred = 0

  print(f"=== Starting evaluation ===")

  for idx, row in data.iterrows():
    text = row["text"]
    correct_label = row["label_word"].lower()
    pred_label = classify(text, labels, model)
    pred_label = pred_label.lower()

    if pred_label:
      if pred_label == correct_label:
        correct_pred += 1
        print(f"Article {idx}: Correctly classified as '{correct_label}'")
      else:
          print(f"Article {idx}: Incorrectly classified. True: '{correct_label}', Predicted: '{pred_label}'")
  acc = (correct_pred / total_samples) * 100
  return acc


In [ ]:
# Test the model
labels = ["World", "Sports", "Business", "Sci/Tech"]

model = "cross-encoder/nli-deberta-v3-large"

print("Evaluating LLM on Balanced Data...")
balanced_accuracy = evaluate(balanced_data, labels, model)
print(f"\nAccuracy on Balanced Data: {balanced_accuracy:.2f}%")

print("\n\nEvaluating LLM on Imbalanced Data...")
imbalanced_accuracy = evaluate(imbalanced_data, labels, model)
print(f"\nAccuracy on Imbalanced Data: {imbalanced_accuracy:.2f}%")

Evaluating LLM on Balanced Data...
=== Starting evaluation ===


Device set to use cuda:0


Article 0: Incorrectly classified. True: 'world', Predicted: 'business'


Device set to use cuda:0


Article 1: Correctly classified as 'sports'


...

Article 496: Correctly classified as 'sports'


Device set to use cuda:0


Article 497: Correctly classified as 'business'


Device set to use cuda:0


Article 498: Correctly classified as 'business'
